[View in Colaboratory](https://colab.research.google.com/github/Hoiy/kaggle-santander-value-prediction-challenge/blob/master/model_nn.ipynb)

In [3]:
!mkdir prep
!gsutil rsync gs://{os.environ['GCP_BUCKET']}/prep prep

Building synchronization state...
Starting synchronization...
Copying gs://kaggle-195720-santander-value-prediction-challenge/prep/test.feather...
Copying gs://kaggle-195720-santander-value-prediction-challenge/prep/train.feather...
|
Operation completed over 2 objects/2.0 GiB.                                      


In [4]:
import pandas as pd
import numpy as np
import dotenv
from scipy import sparse
import pickle
import os

dotenv.load_dotenv('.env')

True

In [0]:
train = pd.read_feather('./prep/train.feather')

In [73]:
feat_cols = train.columns[-5:]
feat_cols

Index(['mean', 'max', 'diff', 'var', 'nz_count'], dtype='object')

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train[feat_cols], train['target'], test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((3567, 5), (892, 5))

In [125]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten, Activation, Reshape, Add, Average
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, Conv1D, Reshape, MaxPooling1D, Concatenate, Dot
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2, l1
from keras.constraints import non_neg, unit_norm
import keras.backend as K
from keras.metrics import mse
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor

def rmse(y_true, y_pred):
  # bug when K.sqrt(mse(y_true, y_pred))
  return K.sqrt(mse(y_true, y_pred))
  # return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(y_true, 1)), tf.log(tf.add(y_pred,1))))))
  
  
def build_model():
    inp = Input(shape=(X_train.shape[1], ))

    emb = Dense(64, activation='relu')(inp)
    emb = Dense(64, activation='relu')(emb)
    out = Dense(1, activation='elu')(emb)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='mse',
                  metrics=[rmse],
                  optimizer='adam')
    
    return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_50 (InputLayer)        (None, 5)                 0         
_________________________________________________________________
dense_138 (Dense)            (None, 64)                384       
_________________________________________________________________
dense_139 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_140 (Dense)            (None, 1)                 65        
Total params: 4,609
Trainable params: 4,609
Non-trainable params: 0
_________________________________________________________________


In [126]:
# val_period = ((train['activation_date'] >= '2017-03-15') & (train['activation_date'] <= '2017-03-16')) |\
#   ((train['activation_date'] >= '2017-03-22') & (train['activation_date'] <= '2017-03-23'))
# train_period = ~val_period

BATCH_SIZE = 32
EPOCHS = 20000
FILE_PATH = './nn_model/nn.dat'
!mkdir nn_model

checkpoint = ModelCheckpoint(FILE_PATH, monitor='val_rmse', save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=10)
callbacks_list = [checkpoint, early] #early

mkdir: cannot create directory ‘nn_model’: File exists


In [0]:
# from sklearn.model_selection import KFold, cross_val_score


# k_fold = KFold(n_splits=5, shuffle=True)
# model = KerasRegressor(build_model, batch_size=BATCH_SIZE)
# metrics = cross_val_score(
#     model, 
#     [train[col] for col in cat_cols] + [train['price_std']], 
#     train['deal_probability'], 
#     cv=k_fold,
#     n_jobs=1,
#     fit_params={
#         'validation_split': 0.2,
#         'shuffle': True,
#         'batch_size': BATCH_SIZE,
#         'epochs': EPOCHS,
#         'callbacks': callbacks_list
#     }
# )

In [128]:
# Full training
# train_inp = [train[train_period][col] for col in cat_cols] + [train[train_period][col] for col in cont_cols]
# val_inp = [train[val_period][col] for col in cat_cols] + [train[val_period][col] for col in cont_cols]

# BATCH_SIZE = 32

# model = KerasRegressor(build_model, batch_size=BATCH_SIZE)
# model.fit(train_inp, train[train_period]['deal_probability'], **{
#         'validation_data': (val_inp, train[val_period]['deal_probability']),
#         'shuffle': True,
#         'batch_size': BATCH_SIZE,
#         'epochs': EPOCHS,
#         'callbacks': callbacks_list
# })
model = KerasRegressor(build_model, batch_size=BATCH_SIZE)
model.fit(X_train, y_train, **{
        'validation_split': 0.2,
        'shuffle': True,
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
        'callbacks': callbacks_list
})

Train on 2853 samples, validate on 714 samples
Epoch 1/20000
2853/2853 [==============================] - 2s 726us/step - loss: 208.8817 - rmse: 14.3414 - val_loss: 199.1402 - val_rmse: 14.0062
Epoch 2/20000
2853/2853 [==============================] - 1s 210us/step - loss: 186.1297 - rmse: 13.5227 - val_loss: 169.0944 - val_rmse: 12.8891
Epoch 3/20000
2853/2853 [==============================] - 1s 210us/step - loss: 149.8188 - rmse: 12.0974 - val_loss: 127.7009 - val_rmse: 11.1685
Epoch 4/20000
2853/2853 [==============================] - 1s 211us/step - loss: 106.4756 - rmse: 10.1465 - val_loss: 84.1738 - val_rmse: 9.0115
Epoch 5/20000
2853/2853 [==============================] - 1s 212us/step - loss: 65.7078 - rmse: 7.8788 - val_loss: 47.5954 - val_rmse: 6.6805
Epoch 6/20000
2853/2853 [==============================] - 1s 213us/step - loss: 34.7587 - rmse: 5.5910 - val_loss: 23.0572 - val_rmse: 4.4823
Epoch 7/20000
2853/2853 [==============================] - 1s 209us/step - loss: 

2853/2853 [==============================] - 1s 210us/step - loss: 3.1821 - rmse: 1.4853 - val_loss: 2.9739 - val_rmse: 1.4213
Epoch 12/20000
2853/2853 [==============================] - 1s 209us/step - loss: 3.1546 - rmse: 1.4685 - val_loss: 2.9661 - val_rmse: 1.4116
Epoch 13/20000
2853/2853 [==============================] - 1s 215us/step - loss: 3.1503 - rmse: 1.4620 - val_loss: 2.9659 - val_rmse: 1.4105
Epoch 14/20000
2853/2853 [==============================] - 1s 214us/step - loss: 3.1495 - rmse: 1.4616 - val_loss: 2.9659 - val_rmse: 1.4087
Epoch 15/20000
2853/2853 [==============================] - 1s 211us/step - loss: 3.1499 - rmse: 1.4586 - val_loss: 2.9658 - val_rmse: 1.4091
Epoch 16/20000
2853/2853 [==============================] - 1s 211us/step - loss: 3.1497 - rmse: 1.4607 - val_loss: 2.9659 - val_rmse: 1.4084
Epoch 17/20000
2853/2853 [==============================] - 1s 213us/step - loss: 3.1492 - rmse: 1.4602 - val_loss: 2.9661 - val_rmse: 1.4076
Epoch 18/20000
2853/2

2853/2853 [==============================] - 1s 210us/step - loss: 3.1500 - rmse: 1.4590 - val_loss: 2.9659 - val_rmse: 1.4108
Epoch 22/20000
2853/2853 [==============================] - 1s 210us/step - loss: 3.1501 - rmse: 1.4629 - val_loss: 2.9659 - val_rmse: 1.4084
Epoch 23/20000
2853/2853 [==============================] - 1s 214us/step - loss: 3.1498 - rmse: 1.4584 - val_loss: 2.9662 - val_rmse: 1.4072
Epoch 24/20000
2853/2853 [==============================] - 1s 210us/step - loss: 3.1497 - rmse: 1.4587 - val_loss: 2.9663 - val_rmse: 1.4120
Epoch 25/20000
2853/2853 [==============================] - 1s 212us/step - loss: 3.1495 - rmse: 1.4617 - val_loss: 2.9663 - val_rmse: 1.4069
Epoch 26/20000
2853/2853 [==============================] - 1s 209us/step - loss: 3.1498 - rmse: 1.4614 - val_loss: 2.9667 - val_rmse: 1.4064
Epoch 27/20000
2853/2853 [==============================] - 1s 212us/step - loss: 3.1504 - rmse: 1.4588 - val_loss: 2.9659 - val_rmse: 1.4107
Epoch 28/20000
2853/2

In [108]:
# from keras.models import load_model
# model = load_model(FILE_PATH, custom_objects={'rmse': rmse})

metric = model.score(X_test, y_test)
metric

892/892 [==============================] - 0s 91us/step


-2.877417519488142

In [0]:
test = pd.read_csv('./raw_data/test.csv')

In [0]:
SUBMISSION_FILE='baseline_nn.csv'
SUBMISSION_MESSAGE='"test %f"'%metric

test['target'] = model.predict(
    test[1:],
    batch_size=BATCH_SIZE
)
test[['ID', 'target']].to_csv(SUBMISSION_FILE, index=False)

In [0]:
len(test['item_id']) == len(submission['item_id'])

True

In [0]:
!kaggle competitions submit -f '{SUBMISSION_FILE}' -m '{SUBMISSION_MESSAGE}'

Using competition: avito-demand-prediction
Successfully submitted to Avito Demand Prediction Challenge

In [0]:
!kaggle competitions submit

usage: kaggle competitions submit [-h] [-c COMPETITION] -f FILE_NAME -m
                                  MESSAGE [-q]
kaggle competitions submit: error: the following arguments are required: -f/--file, -m/--message


In [0]:
!echo "{SUBMISSION_FILE}"

baseline.csv


In [0]:
!kaggle competitions submit -m 'Baseline -0.22988' -f baseline.csv

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 11, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/kaggle/cli.py", line 48, in main
    out = args.func(**command_args)
TypeError: competition_submit_cli() got an unexpected keyword argument 'file_name'


In [0]:
!pip freeze | grep kaggle

kaggle==1.3.9


In [0]:
!kaggle --version

Kaggle API 1.3.8
